In [1]:
from tensorflow.keras.datasets import cifar10

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Lambda, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.applications.vgg19 import decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.models import Model
import cv2
from keras_preprocessing.image import ImageDataGenerator
import scipy.io
import tensorflow as tf
import json
import sys
sys.path.append('..')  
from GradPri_utils.utils import *

from tensorflow.keras.losses import categorical_crossentropy
from progressbar import ProgressBar
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# tf.compat.v1.disable_eager_execution()

In [2]:
DATA_PATH = '/public/liujiawei/ZHB/GradPri/UdacityData'

X_train = np.load(os.path.join(DATA_PATH, 'train_X.npy'))
Y_train = np.load(os.path.join(DATA_PATH, 'train_Y.npy'))
X_test = np.load(os.path.join(DATA_PATH, 'test_X.npy'))
Y_test = np.load(os.path.join(DATA_PATH, 'test_Y.npy'))

In [3]:
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# X_train /= 255
# X_test /= 255

x_train shape: (33808, 128, 128, 3)
33808 train samples
5279 test samples


In [4]:
input_shape = (128, 128, 3)

model = Sequential()
# model.add(Lambda(augment_2d,
#                  input_shape=x_train.shape[1:],
#                  arguments={'rotation': 8.0, 'horizontal_flip': True}))
# Block 1
model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv1', input_shape=input_shape))
model.add(BatchNormalization(name = 'block1_bn1'))
model.add(Activation('relu', name = 'block1_act1'))

model.add(Conv2D(64, (3, 3), padding='same', name = 'block1_conv2'))
model.add(BatchNormalization(name = 'block1_bn2'))
model.add(Activation('relu', name = 'block1_act2'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block1_pool'))
model.add(Dropout(0.25, name='block1_drop'))

# Block 2
model.add(Conv2D(128, (3, 3), padding='same', name = 'block2_conv1'))
model.add(BatchNormalization(name = 'block2_bn1'))
model.add(Activation('relu', name = 'block2_act1'))

model.add(Conv2D(128, (3, 3), padding='same', name = 'block2_conv2'))
model.add(BatchNormalization(name = 'block2_bn2'))
model.add(Activation('relu', name = 'block2_act2'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block2_pool'))
model.add(Dropout(0.25, name='block2_drop'))

# Block 3
model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv1'))
model.add(BatchNormalization(name = 'block3_bn1'))
model.add(Activation('relu', name = 'block3_act1'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv2'))
model.add(BatchNormalization(name = 'block3_bn2'))
model.add(Activation('relu', name = 'block3_act2'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv3'))
model.add(BatchNormalization(name = 'block3_bn3'))
model.add(Activation('relu', name = 'block3_act3'))

model.add(Conv2D(256, (3, 3), padding='same', name = 'block3_conv4'))
model.add(BatchNormalization(name = 'block3_bn4'))
model.add(Activation('relu', name = 'block3_act4'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block3_pool'))
model.add(Dropout(0.25, name='block3_drop'))

# Block 4
model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv1'))
model.add(BatchNormalization(name = 'block4_bn1'))
model.add(Activation('relu', name = 'block4_act1'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv2'))
model.add(BatchNormalization(name = 'block4_bn2'))
model.add(Activation('relu', name = 'block4_act2'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv3'))
model.add(BatchNormalization(name = 'block4_bn3'))
model.add(Activation('relu', name = 'block4_act3'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block4_conv4'))
model.add(BatchNormalization(name = 'block4_bn4'))
model.add(Activation('relu', name = 'block4_act4'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block4_pool'))
model.add(Dropout(0.25, name='block4_drop'))

# Block 5
model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv1'))
model.add(BatchNormalization(name = 'block5_bn1'))
model.add(Activation('relu', name = 'block5_act1'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv2'))
model.add(BatchNormalization(name = 'block5_bn2'))
model.add(Activation('relu', name = 'block5_act2'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv3'))
model.add(BatchNormalization(name = 'block5_bn3'))
model.add(Activation('relu', name = 'block5_act3'))

model.add(Conv2D(512, (3, 3), padding='same', name = 'block5_conv4'))
model.add(BatchNormalization(name = 'block5_bn4'))
model.add(Activation('relu', name = 'block5_act4'))

model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block5_pool'))
model.add(Dropout(0.25, name='block5_drop'))

# include_top
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dropout(0.5, name='drop1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dropout(0.5, name='drop2'))
model.add(Dense(1, activation='tanh', name='predictions'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 128, 128, 64)      256       
_________________________________________________________________
block1_act1 (Activation)     (None, 128, 128, 64)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 128, 128, 64)      256       
_________________________________________________________________
block1_act2 (Activation)     (None, 128, 128, 64)      0         
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0

In [11]:
model = load_model("./model/Udacity_VGG19.h5")
# # initiate RMSprop optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.00004) # 0.00005

# # Let's train the model using RMSprop
model.compile(loss='mse',
              optimizer=opt,
              metrics=['mse'])
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="./model/Udacity_VGG19.h5", 
                                                monitor='val_mse', 
                                                verbose=1, save_best_only=True, mode = 'min')
callbacks_list = [checkpoint]

model.fit(X_train, Y_train, batch_size=128, epochs=50, shuffle=True,
          validation_data=(X_test[0:2000], Y_test[0:2000]), callbacks=callbacks_list)

Epoch 1/50
265/265 [==============================] - 61s 224ms/step - loss: 0.0180 - mse: 0.0180 - val_loss: 0.0397 - val_mse: 0.0397

Epoch 00001: val_mse improved from inf to 0.03974, saving model to ./model/Udacity_VGG19.h5
Epoch 2/50
265/265 [==============================] - 58s 218ms/step - loss: 0.0146 - mse: 0.0146 - val_loss: 0.0171 - val_mse: 0.0171

Epoch 00002: val_mse improved from 0.03974 to 0.01712, saving model to ./model/Udacity_VGG19.h5
Epoch 3/50
265/265 [==============================] - 58s 220ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.0181 - val_mse: 0.0181

Epoch 00003: val_mse did not improve from 0.01712
Epoch 4/50
265/265 [==============================] - 58s 219ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.0172 - val_mse: 0.0172

Epoch 00004: val_mse did not improve from 0.01712
Epoch 5/50
265/265 [==============================] - 55s 208ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.0219 - val_mse: 0.0219

Epoch 00005: val_mse did not imp

In [12]:
# model = load_model("./model/Udacity_VGG19.h5")
pred_tmp = model.predict(X_test)

In [32]:
rmse = 0
for i in range(len(pred_tmp)):
    rmse = rmse + pow(pred_tmp[i][0] - Y_test[i], 2)
rmse = rmse/len(pred_tmp)
rmse = np.sqrt(rmse)
print(rmse)

0.12586020573124734


In [40]:
rmse = []
for i in range(len(pred_tmp)):
    rmse.append( np.sum(pow( pred_tmp[i][0] - Y_test[i], 2)))
rmse1 = np.mean(rmse)
# rmse1 = np.sqrt(rmse1)
ss = 0
for i in rmse:
    if i<rmse1:
        ss += 1
print(ss, rmse1)
# print(rmse1, rmse)

4187 0.015840791386711836


In [14]:
metric_tmp = model.evaluate(X_test, Y_test)
print(metric_tmp)

165/165 [==============================] - 3s 20ms/step - loss: 0.0158 - mse: 0.0158
[0.015840787440538406, 0.015840787440538406]


In [15]:
rmse = 0
for i in range(len(pred_tmp)):
    rmse = rmse + pow(0 - Y_test[i], 2)
rmse = rmse/len(pred_tmp)
rmse = np.sqrt(rmse)
print(rmse)

0.2131508352233873
